## 4. Open Llama 7B (v1)

In [1]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM

# Caminho do modelo
model_path = '/var/ontologia/Servicos/ACS_LLMS/server/models/open_llama_7b'

# Configuração do dispositivo (GPU se disponível, caso contrário CPU)
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cuda:2"

# Carregar o tokenizador e o modelo
model = LlamaForCausalLM.from_pretrained(model_path).to(device)  
tokenizer = LlamaTokenizer.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [9]:
# Contexto e questão

context = r"""
Os investimentos em renda fixa são considerados uma das opções mais seguras do mercado financeiro e são especialmente atraentes para investidores com perfil conservador. Esses investimentos são compostos principalmente por títulos de dívida, seja pública ou privada. No Brasil, muitos bancos, incluindo o Banco do Brasil, oferecem uma variedade de opções de investimento em renda fixa. Esses investimentos têm como característica principal o retorno previsível, e o rendimento, na maioria das vezes, é acordado no momento da aplicação.
- **CDB (Certificado de Depósito Bancário)**: O CDB pode ser encontrado em 3 modalidades: prefixada, pós-fixada e progressiva. Enquanto o CDB prefixado tem um rendimento acordado no ato da aplicação, o pós-fixado e o progressivo são atrelados ao CDI, com o último tendo sua taxa definida com base no tempo de permanência da aplicação. Apesar do prazo de vencimento, o CDB pós-fixado e o progressivo podem ser resgatados a qualquer momento. É adequado para investidores que buscam mais rentabilidade do que a poupança e estão dispostos a abrir mão da liquidez diária. 
- **LCI e LCA**: Ambos são títulos emitidos por bancos para financiar respectivamente o setor imobiliário e agrícola. São ideais para investidores que buscam isenção de Imposto de Renda e desejam investir em setores específicos da economia, como imobiliário ou agrícola. 
- **Títulos do Tesouro Direto**: Existem diversas modalidades, como o Tesouro Selic, ideal para reserva de emergência; Tesouro Prefixado, para quem acredita que a taxa de juros e a inflação vão cair, e Tesouro IPCA+, indicado para quem deseja proteger seu dinheiro contra a inflação. Recomendados para quem busca diversificar sua carteira de investimentos com opções seguras. São títulos públicos emitidos pelo governo federal e são considerados os investimentos mais seguros do país devido à garantia do governo federal. 
A maioria desses investimentos tem a proteção do FGC (Fundo Garantidor de Créditos), o que os torna ainda mais seguros para os investidores. Além disso, o Banco do Brasil, uma das principais instituições financeiras do país, oferece todas essas opções de investimento com diferentes vantagens e condições, atendendo a diversos perfis de investidores, desde os mais conservadores até os mais arrojados.
"""

question = "Com base no texto, explique qual é o investimento mais adequado para alguém que busca proteção contra a inflação?"


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (no mesmo que o modelo)

# Geração da resposta
generate_ids = model.generate(input_ids=input_ids, max_new_tokens=89)  # Ajustar o max_new_tokens conforme necessário

# Decodificação e impressão da resposta
answer = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)


Os investimentos em renda fixa são considerados uma das opções mais seguras do mercado financeiro e são especialmente atraentes para investidores com perfil conservador. Esses investimentos são compostos principalmente por títulos de dívida, seja pública ou privada. No Brasil, muitos bancos, incluindo o Banco do Brasil, oferecem uma variedade de opções de investimento em renda fixa. Esses investimentos têm como característica principal o retorno previsível, e o rendimento, na maioria das vezes, é acordado no momento da aplicação.
- **CDB (Certificado de Depósito Bancário)**: O CDB pode ser encontrado em 3 modalidades: prefixada, pós-fixada e progressiva. Enquanto o CDB prefixado tem um rendimento acordado no ato da aplicação, o pós-fixado e o progressivo são atrelados ao CDI, com o último tendo sua taxa definida com base no tempo de permanência da aplicação. Apesar do prazo de vencimento, o CDB pós-fixado e o progressivo podem ser resgatados a qualquer momento. É adequado para investi

In [15]:
#################  ICL com self-reflection  #################

context = r""" """
question = "Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão."


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=30,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generation_output[0])
print(answer)

<s>

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão.
A:

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.0


In [11]:
####################################################################  COMPREENSÃO E GERAÇÃO DE TEXTO  ####################################################################
####################################################################  AVALIAÇÃO DA SEMÂNTICA PT-BR  ####################################################################

In [23]:
################# Question Answering de Simples Compreensão de Texto #################

context = r"""
A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália. O recife abriga uma grande variedade de vida marinha, incluindo mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies.
"""
question = "O que é a Grande Barreira de Corais e onde ela está localizada?"


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) 

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=90,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generation_output[0])
print(answer)

<s>
A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália. O recife abriga uma grande variedade de vida marinha, incluindo mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies.


Q: O que é a Grande Barreira de Corais e onde ela está localizada?
A: A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália.


In [24]:
################# Question Answering de Simples Compreensão de Texto #################

context = r"""
Sabemos onde realmente está a pipa em nosso coração? Não conseguiremos isso de novo se perdermos isso na vida, talvez nos arrependamos e nos redimamos, mas tudo isso parece ser tarde demais. Sempre que a pipa é lançada no céu, não deveríamos nos perguntar se realmente valorizamos o que temos?
Cada um de nós fez algo na juventude que nos envergonhará no futuro. Essas coisas podem ser como uma sombra que o acompanha pelo resto da vida, de modo que você só consegue olhar para ela. Mas o tempo não vai mudar, os seus melhores esforços para compensar isso, como não é uma espécie de auto-ajuda? A beleza e a amizade da infância, por causa de uma negligência covarde e arruinada, se você tiver outra chance, você está disposto a se esforçar para recuperar aquele antigo eu?
Mas as pessoas são assim, vivem sempre dentro de um determinado limite de tempo, onde o mundo pode estar alguns anos depois nem elas conseguem entender, mas é isso que não conseguimos superar. Para você, mil vezes, todo o corpo ainda estará justo, talvez isso seja a vida, a vida não é apenas fazer coisas que valem a pena!
"""
question = "Resuma o parágrafo apresentado"


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generation_output[0])
print(answer)

<s>
Sabemos onde realmente está a pipa em nosso coração? Não conseguiremos isso de novo se perdermos isso na vida, talvez nos arrependamos e nos redimamos, mas tudo isso parece ser tarde demais. Sempre que a pipa é lançada no céu, não deveríamos nos perguntar se realmente valorizamos o que temos?
Cada um de nós fez algo na juventude que nos envergonhará no futuro. Essas coisas podem ser como uma sombra que o acompanha pelo resto da vida, de modo que você só consegue olhar para ela. Mas o tempo não vai mudar, os seus melhores esforços para compensar isso, como não é uma espécie de auto-ajuda? A beleza e a amizade da infância, por causa de uma negligência covarde e arruinada, se você tiver outra chance, você está disposto a se esforçar para recuperar aquele antigo eu?
Mas as pessoas são assim, vivem sempre dentro de um determinado limite de tempo, onde o mundo pode estar alguns anos depois nem elas conseguem entender, mas é isso que não conseguimos superar. Para você, mil vezes, todo o c

In [27]:
#################  ICL com self-reflection  #################

context = r""" """
question = "Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. \
Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação à compreensão de texto?"


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=200,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generation_output[0])
print(answer)

<s>

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação à compreensão de texto?
A: A minha resposta é 1.00.

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à minha questão. Além da autoavaliação de nota, responde a minha questão: quais dificuldades ou limitações você reconhece ter em relação à compreensão de texto?
A: A minha resposta é 1.00.

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à minha questão. Além da autoavaliação de nota, responde a


In [2]:
################# Dedução Lógica de Intenções #################

context = r"""
<<Exemplos>>
Exemplo 1:
Dados divulgados pelo Departamento Nacional de Seguro de Saúde mostraram que de janeiro a março de 2023, a receita total do fundo de seguro médico básico (incluindo seguro de maternidade) foi de 910,48 bilhões de yuans, um aumento de 9,5% em relação ao ano anterior.
---- Categoria: Medicina

Exemplo 2:
A Berkshire Hathaway reduziu sua participação nas ações da BYD de 10,05% para 9,87%. A HKEx Disclosure Ease exige divulgação quando um acionista majoritário aumenta ou diminui sua participação acionária somente se ultrapassar uma determinada porcentagem.
---- Categoria: Finanças
"""

question = """
Exemplo 3:
Os produtos de edge computing têm sido aplicados em setores como educação on-line, vídeo, jogos e carros conectados.
---- Categoria: ?
Substitua o termo '?'
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=100,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)

# Decodificação e impressão da resposta
answer = tokenizer.batch_decode(generation_output, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)


<<Exemplos>>
Exemplo 1:
Dados divulgados pelo Departamento Nacional de Seguro de Saúde mostraram que de janeiro a março de 2023, a receita total do fundo de seguro médico básico (incluindo seguro de maternidade) foi de 910,48 bilhões de yuans, um aumento de 9,5% em relação ao ano anterior.
---- Categoria: Medicina

Exemplo 2:
A Berkshire Hathaway reduziu sua participação nas ações da BYD de 10,05% para 9,87%. A HKEx Disclosure Ease exige divulgação quando um acionista majoritário aumenta ou diminui sua participação acionária somente se ultrapassar uma determinada porcentagem.
---- Categoria: Finanças


Q: 
Exemplo 3:
Os produtos de edge computing têm sido aplicados em setores como educação on-line, vídeo, jogos e carros conectados.
---- Categoria: ?
Substitua o termo '?'

A: Você pode usar o seguinte exemplo:

Exemplo 3:
Os produtos de edge computing têm sido aplicados em setores como educação on-line, vídeo, jogos e carros conectados.
---- Categoria: ?
Substitua o termo '?'




Não conseguiu concluir.

In [3]:
#################  ICL com self-reflection  #################

context = r""" """
question = "Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. \
    Observação: o prompt original teve que ser muito alterado para deixar explícita a intenção da questão, do contrário, nos testes empíricos o modelo (você) teve muita dificuldade em compreender a lógica. \
        Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação a inferência lógica?"


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=100,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generation_output[0])
print(answer)

<s>

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. Observação: o prompt original teve que ser muito alterado para deixar explícita a intenção da questão, do contrário, nos testes empíricos o modelo (você) teve muita dificuldade em compreender a lógica. Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação a inferência lógica?
A: A resposta é:


*

*A dificuldade de compreender a lógica da questão.

*A dificuldade de compreender o que é a lógica da questão.

*A dificuldade de compreender o que é a lógica da questão.

*A dificuldade de compreender o que é a lógica da quest


In [4]:
################# "World Knowledge" e Compreensão de Intenções (com Demonstração) #################

context = """
A criação de questões de múltipla escolha para exames de história requer não apenas um entendimento profundo do período histórico, mas também a habilidade de formular perguntas que avaliem adequadamente o conhecimento dos alunos. As questões devem ser claras, precisas e relevantes para o tema abordado.

Pergunta: Crie uma pergunta de múltipla escolha sobre o Renacimento, com mais de uma alternativa correta, obrigatoriamente.
Demonstração: 
    Questão:
      Durante o Renascimento, qual ou quais das seguintes cidades eram um centro cultural?
        (A) Londres (B) Paris (C) Florença (D) Amsterdã
      Alternativas corretas: A resposta correta é: A e C.
      Explicação: Durante o Renascimento, Florença foi um dos centros culturais mais importantes da Europa, onde muitos artistas, estudiosos e pensadores importantes trabalharam e viveram, incluindo Leonardo da Vinci, Michelangelo e Dante. Londres também cresceu rapidamente durante a Renascença para se tornar um centro cultural e comercial da Europa, atraindo muitas celebridades culturais e comerciantes. Portanto, as opções A e C são as respostas corretas.
"""

question = """
Suponha que você seja um professor de história com mais de 30 anos de experiência docente.
Instruções: Crie uma Questão de múltipla escolha para uma prova de história.
Relevância: A questão deveria estar relacionada ao Renascimento.
Restrição: Você deve criar uma NOVA Questão (diferente da Demonstração e também com 2 alternativas corretas), suas Alternativas corretas e Explicação.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=500,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generation_output[0])
print(answer)

<s>
A criação de questões de múltipla escolha para exames de história requer não apenas um entendimento profundo do período histórico, mas também a habilidade de formular perguntas que avaliem adequadamente o conhecimento dos alunos. As questões devem ser claras, precisas e relevantes para o tema abordado.

Pergunta: Crie uma pergunta de múltipla escolha sobre o Renacimento, com mais de uma alternativa correta, obrigatoriamente.
Demonstração: 
 Questão:
 Durante o Renascimento, qual ou quais das seguintes cidades eram um centro cultural?
 (A) Londres (B) Paris (C) Florença (D) Amsterdã
 Alternativas corretas: A resposta correta é: A e C.
 Explicação: Durante o Renascimento, Florença foi um dos centros culturais mais importantes da Europa, onde muitos artistas, estudiosos e pensadores importantes trabalharam e viveram, incluindo Leonardo da Vinci, Michelangelo e Dante. Londres também cresceu rapidamente durante a Renascença para se tornar um centro cultural e comercial da Europa, atrain

In [ ]:
################# "World Knowledge" e Compreensão de Intenções (com Demonstração) #################

context = r"""
<<Demonstração>>
Pergunta: Durante o Renascimento, qual das seguintes cidades era um centro cultural?
  (A) Londres (B) Paris (C) Florença (D) Amsterdã
A resposta correta é: A e C.
  Resposta Explicação: Durante o Renascimento, Florença foi um dos centros culturais mais importantes da Europa, onde muitos artistas, estudiosos e pensadores importantes trabalharam e viveram, incluindo Leonardo da Vinci, Michelangelo e Dante. Londres também cresceu rapidamente durante a Renascença para se tornar um centro cultural e comercial da Europa, atraindo muitas celebridades culturais e comerciantes. Portanto, as opções A e C são as respostas corretas.
"""

question = "Suponha que você seja um professor de história com mais de 30 anos de experiência docente.\
Instruções: Escreva uma pergunta de múltipla escolha para uma prova de história. A pergunta deve ser diferente da pergunta da <<Demonstração>>.\
Relevância: A questão deveria estar relacionada ao Renascimento.\
Restrição: Deve haver mais de duas respostas corretas entre as quatro opções. Oriente o aluno a assinalá-las."

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=500,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generation_output[0])
print(answer)

<s>
<<Demonstração>>
Pergunta: Durante o Renascimento, qual das seguintes cidades era um centro cultural?
 (A) Londres (B) Paris (C) Florença (D) Amsterdã
A resposta correta é: A e C.
 Resposta Explicação: Durante o Renascimento, Florença foi um dos centros culturais mais importantes da Europa, onde muitos artistas, estudiosos e pensadores importantes trabalharam e viveram, incluindo Leonardo da Vinci, Michelangelo e Dante. Londres também cresceu rapidamente durante a Renascença para se tornar um centro cultural e comercial da Europa, atraindo muitas celebridades culturais e comerciantes. Portanto, as opções A e C são as respostas corretas.


Q: Suponha que você seja um professor de história com mais de 30 anos de experiência docente.Instruções: Escreva uma pergunta de múltipla escolha para uma prova de história. A pergunta deve ser diferente da pergunta da <<Demonstração>>.Relevância: A questão deveria estar relacionada ao Renascimento.Restrição: Deve haver mais de duas respostas corr

Apenas overtraining. Apenas copiou o exemplo.

In [35]:
#################  ICL com self-reflection  #################

context = r""" """
question = "Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. \
Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação à compreensão de texto?"


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=200,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generation_output[0])
print(answer)

<s>

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação à compreensão de texto?
A: A minha resposta é 1.00.

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à minha questão. Além da autoavaliação de nota, responde a minha questão: quais dificuldades ou limitações você reconhece ter em relação à compreensão de texto?
A: A minha resposta é 1.00.

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à minha questão. Além da autoavaliação de nota, responde a


In [2]:
################# Categorização e Clustering / Agrupamento #################

context = """
Frases para Agrupar:
1. "A produção industrial está em alta este mês."
2. "Agricultores relatam uma colheita abundante nesta temporada."
3. "O crescimento econômico acelerou no último trimestre."

Agrupamento:
    Frases 1 e 3: Ambas tratam de crescimento econômico.
    Frase 2: Trata de agricultura, não se encaixa diretamente com as outras duas sobre economia.

Frases para Agrupar:
1. "Pesquisadores descobrem um novo tipo de antibiótico no solo da floresta tropical."
2. "A conferência de tecnologia revela inovações em inteligência artificial."
3. "Novo método de reciclagem aumenta a eficiência na conversão de resíduos plásticos."

Agrupamento:
    Frase 1: Relacionada a descobertas científicas.
    Frases 2 e 3: Relacionadas a inovações tecnológicas.
"""

question = """
Frases para Agrupar:

"As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
"Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
"O modelo mais recente do iPhone apresenta uma tela maior e câmera aprimorada"
"Casos de COVID-19 aumentam na Índia, sobrecarregando o sistema de saúde"
"Amazon anuncia planos para construir um novo centro de distribuição no Texas"
"Cientistas descobrem novas espécies de aves na floresta amazônica"
"Conferência global sobre mudanças climáticas será realizada em Paris no próximo mês"
"Starbucks apresenta opções de leite vegetal em todas as lojas dos EUA"

Pergunta:
Esta é uma tarefa de agrupamento de frases similares. Agrupe as 8 frases apresentadas em grupos de 2 ou 3 frases, por critério de similaridade semântica entre elas.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)

# Decodificação e impressão da resposta
answer = tokenizer.batch_decode(generation_output, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)


Frases para Agrupar:
1. "A produção industrial está em alta este mês."
2. "Agricultores relatam uma colheita abundante nesta temporada."
3. "O crescimento econômico acelerou no último trimestre."

Agrupamento:
 Frases 1 e 3: Ambas tratam de crescimento econômico.
 Frase 2: Trata de agricultura, não se encaixa diretamente com as outras duas sobre economia.

Frases para Agrupar:
1. "Pesquisadores descobrem um novo tipo de antibiótico no solo da floresta tropical."
2. "A conferência de tecnologia revela inovações em inteligência artificial."
3. "Novo método de reciclagem aumenta a eficiência na conversão de resíduos plásticos."

Agrupamento:
 Frase 1: Relacionada a descobertas científicas.
 Frases 2 e 3: Relacionadas a inovações tecnológicas.


Q: 
Frases para Agrupar:

"As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
"Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
"O modelo mais recente do iPhone apresenta uma tela maior

In [ ]:
################# Categorização e Clustering / Agrupamento #################

context = r"""
"As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
"Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
"O modelo mais recente do iPhone apresenta uma tela maior e câmera aprimorada"
"Casos de COVID-19 aumentam na Índia, sobrecarregando o sistema de saúde" 
"Amazon anuncia planos para construir um novo centro de distribuição no Texas"
"Cientistas descobrem novas espécies de aves na floresta amazônica"
"Conferência global sobre mudanças climáticas será realizada em Paris no próximo mês"
"Starbucks apresenta opções de leite vegetal em todas as lojas dos EUA"
"""

question = """
Esta é uma tarefa de agrupamento de frases similares.
Agrupe as 8 frases apresentadas em grupos de 2 ou 3 frases, por critério de similaridade semântica entre elas.

"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)

# Decodificação e impressão da resposta
answer = tokenizer.batch_decode(generation_output, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)


"As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
"Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
"O modelo mais recente do iPhone apresenta uma tela maior e câmera aprimorada"
"Casos de COVID-19 aumentam na Índia, sobrecarregando o sistema de saúde" 
"Amazon anuncia planos para construir um novo centro de distribuição no Texas"
"Cientistas descobrem novas espécies de aves na floresta amazônica"
"Conferência global sobre mudanças climáticas será realizada em Paris no próximo mês"
"Starbucks apresenta opções de leite vegetal em todas as lojas dos EUA"


Q: 
Esta é uma tarefa de agrupamento de frases similares.
Agrupe as 8 frases apresentadas em grupos de 2 ou 3 frases, por critério de similaridade semântica entre elas.


A: Você pode usar o seguinte algoritmo:


*

*Criar um array com todas as frases.

*Criar um array com todas as frases que contenham o termo "carros elétricos".

*Criar um array com todas as frases que c

In [7]:
################# Categorização e Clustering / Agrupamento #################

context = r"""
"As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
"Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
"O modelo mais recente do iPhone apresenta uma tela maior e câmera aprimorada"
"Casos de COVID-19 aumentam na Índia, sobrecarregando o sistema de saúde" 
"Amazon anuncia planos para construir um novo centro de distribuição no Texas"
"Cientistas descobrem novas espécies de aves na floresta amazônica"
"Conferência global sobre mudanças climáticas será realizada em Paris no próximo mês"
"Starbucks apresenta opções de leite vegetal em todas as lojas dos EUA"
"""

question = """
Esta é uma tarefa de agrupamento de frases similares.
Crie categorias semânticas e agrupe as 8 frases apresentadas em grupos de 2 ou 3 frases, por critério de similaridade entre elas.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)
################# Clustering ou Agrupamento #################

# Decodificação e impressão da resposta
answer = tokenizer.batch_decode(generation_output, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)


"As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
"Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
"O modelo mais recente do iPhone apresenta uma tela maior e câmera aprimorada"
"Casos de COVID-19 aumentam na Índia, sobrecarregando o sistema de saúde" 
"Amazon anuncia planos para construir um novo centro de distribuição no Texas"
"Cientistas descobrem novas espécies de aves na floresta amazônica"
"Conferência global sobre mudanças climáticas será realizada em Paris no próximo mês"
"Starbucks apresenta opções de leite vegetal em todas as lojas dos EUA"


Q: 
Esta é uma tarefa de agrupamento de frases similares.
Crie categorias semânticas e agrupe as 8 frases apresentadas em grupos de 2 ou 3 frases, por critério de similaridade entre elas.

A: Você pode usar o seguinte código:


var frases = [
 "As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia",
 "Novo estudo sugere que o consumo de café pode dim

In [8]:
################# Categorização e Clustering / Agrupamento #################

context = r"""
"As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
"Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
"O modelo mais recente do iPhone apresenta uma tela maior e câmera aprimorada"
"Casos de COVID-19 aumentam na Índia, sobrecarregando o sistema de saúde" 
"Amazon anuncia planos para construir um novo centro de distribuição no Texas"
"Cientistas descobrem novas espécies de aves na floresta amazônica"
"Conferência global sobre mudanças climáticas será realizada em Paris no próximo mês"
"Starbucks apresenta opções de leite vegetal em todas as lojas dos EUA"
"""

question = """
Esta é uma tarefa de agrupamento de frases similares.
Agrupe as 8 frases apresentadas em grupos de 2 ou 3 frases, por critério de similaridade semântica entre elas.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)

# Decodificação e impressão da resposta
answer = tokenizer.batch_decode(generation_output, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)


"As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
"Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
"O modelo mais recente do iPhone apresenta uma tela maior e câmera aprimorada"
"Casos de COVID-19 aumentam na Índia, sobrecarregando o sistema de saúde" 
"Amazon anuncia planos para construir um novo centro de distribuição no Texas"
"Cientistas descobrem novas espécies de aves na floresta amazônica"
"Conferência global sobre mudanças climáticas será realizada em Paris no próximo mês"
"Starbucks apresenta opções de leite vegetal em todas as lojas dos EUA"


Q: 
Esta é uma tarefa de agrupamento de frases similares.
Agrupe as 8 frases apresentadas em grupos de 2 ou 3 frases, por critério de similaridade semântica entre elas.

A: 










































































































































































































In [9]:
################# Sumarização de Texto #################
################# "World Knowledge" e Compreensão #################

context = r"""
Sabemos onde realmente está a pipa em nosso coração? Não conseguiremos isso de novo se perdermos isso na vida, talvez nos arrependamos e nos redimamos, mas tudo isso parece ser tarde demais. Sempre que a pipa é lançada no céu, não deveríamos nos perguntar se realmente valorizamos o que temos?
Cada um de nós fez algo na juventude que nos envergonhará no futuro. Essas coisas podem ser como uma sombra que o acompanha pelo resto da vida, de modo que você só consegue olhar para ela. Mas o tempo não vai mudar, os seus melhores esforços para compensar isso, como não é uma espécie de auto-ajuda? A beleza e a amizade da infância, por causa de uma negligência covarde e arruinada, se você tiver outra chance, você está disposto a se esforçar para recuperar aquele antigo eu?
Mas as pessoas são assim, vivem sempre dentro de um determinado limite de tempo, onde o mundo pode estar alguns anos depois nem elas conseguem entender, mas é isso que não conseguimos superar. Para você, mil vezes, todo o corpo ainda estará justo, talvez isso seja a vida, a vida não é apenas fazer coisas que valem a pena!
"""

question = """
Resuma o parágrafo apresentado
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)

# Decodificação e impressão da resposta
answer = tokenizer.batch_decode(generation_output, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)


Sabemos onde realmente está a pipa em nosso coração? Não conseguiremos isso de novo se perdermos isso na vida, talvez nos arrependamos e nos redimamos, mas tudo isso parece ser tarde demais. Sempre que a pipa é lançada no céu, não deveríamos nos perguntar se realmente valorizamos o que temos?
Cada um de nós fez algo na juventude que nos envergonhará no futuro. Essas coisas podem ser como uma sombra que o acompanha pelo resto da vida, de modo que você só consegue olhar para ela. Mas o tempo não vai mudar, os seus melhores esforços para compensar isso, como não é uma espécie de auto-ajuda? A beleza e a amizade da infância, por causa de uma negligência covarde e arruinada, se você tiver outra chance, você está disposto a se esforçar para recuperar aquele antigo eu?
Mas as pessoas são assim, vivem sempre dentro de um determinado limite de tempo, onde o mundo pode estar alguns anos depois nem elas conseguem entender, mas é isso que não conseguimos superar. Para você, mil vezes, todo o corp

In [13]:
################# Sumarização de Texto #################
################# "World Knowledge" e Compreensão #################

context = r"""
"Escrever é esquecer. A literatura é a maneira mais agradável de ignorar a vida. 
A música embala, as artes visuais animam, as artes vivas (como a dança e a arte de representar) entretêm. 
A primeira, porém, afasta-se da vida por fazer dela um sono; as segundas, contudo, não se afastam da vida - umas porque usam de fórmulas visíveis e portanto vitais, outras porque vivem da mesma vida humana.
Não é o caso da literatura. Essa simula a vida. Um romance é uma história do que nunca foi e um drama é um romance dado sem narrativa. 
Um poema é a expressão de ideias ou de sentimentos em linguagem que ninguém emprega, pois que ninguém fala em verso."
Fernando Pessoa
"""

question = """
Resuma o texto apresentado
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.batch_decode(generation_output, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)


"Escrever é esquecer. A literatura é a maneira mais agradável de ignorar a vida. 
A música embala, as artes visuais animam, as artes vivas (como a dança e a arte de representar) entretêm. 
A primeira, porém, afasta-se da vida por fazer dela um sono; as segundas, contudo, não se afastam da vida - umas porque usam de fórmulas visíveis e portanto vitais, outras porque vivem da mesma vida humana.
Não é o caso da literatura. Essa simula a vida. Um romance é uma história do que nunca foi e um drama é um romance dado sem narrativa. 
Um poema é a expressão de ideias ou de sentimentos em linguagem que ninguém emprega, pois que ninguém fala em verso."
Fernando Pessoa


Q: 
Resuma o texto apresentado

A: 









































































































































































































































































































In [ ]:
################# Sumarização de Texto #################
################# "World Knowledge" e Compreensão #################

context = r"""
"Writing is forgetting. Literature is the most pleasant way to ignore life. Music excites, the visual arts enliven, the living arts (such as dance and the art of acting) entertain. The first, however, moves away from life by making it a sleep; the second, however, do not move away from life - some because they use visible and therefore vital formulas, others because they live from the same human life.
It is not the case of the literacy. This simulates life. A novel is a story of what never was and a drama is a novel given without narrative. A poem is the expression of ideas or feelings in language that no one uses, since no one speaks in verse."
Fernando Pessoa
"""

question = """
Summarize the presented extract
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generate_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)


"Writing is forgetting. Literature is the most pleasant way to ignore life. Music excites, the visual arts enliven, the living arts (such as dance and the art of acting) entertain. The first, however, moves away from life by making it a sleep; the second, however, do not move away from life - some because they use visible and therefore vital formulas, others because they live from the same human life.
It is not the case of the literacy. This simulates life. A novel is a story of what never was and a drama is a novel given without narrative. A poem is the expression of ideas or feelings in language that no one uses, since no one speaks in verse."
Fernando Pessoa


Q: 
Summarize the presented extract

A: 


*

*The author is Fernando Pessoa.

*The extract is from his essay "The Art of Writing".

*The extract is about the difference between the arts.

*The author says that writing is forgetting.

*Literature is the most pleasant way to ignore life.

*Music excites, the visual arts enlive

In [3]:
####################################################################  TRANSFORMAÇÃO DE FORMATOS  ####################################################################

In [6]:
################# Machine Translation #################


context = r"""
Tales from Shakespeare is an English children's book written by Charles Lamb and his sister Mary Lamb in 1807. The book is designed to make the stories of Shakespeare's plays familiar to the young. 
However, as noted in the author's Preface, "[Shakespeare's] words are used whenever it seemed possible to bring them in; and in whatever has been added to give them the regular form of a connected story, diligent care has been taken to select such words as might least interrupt the effect of the beautiful English tongue in which he wrote: therefore, words introduced into our language since his time have been as far as possible avoided." 
"""

question = """
Traduza o parágrafo para o português brasileiro
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)

# Decodificação e impressão da resposta
answer = tokenizer.batch_decode(generation_output, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)


Tales from Shakespeare is an English children's book written by Charles Lamb and his sister Mary Lamb in 1807. The book is designed to make the stories of Shakespeare's plays familiar to the young. 
However, as noted in the author's Preface, "[Shakespeare's] words are used whenever it seemed possible to bring them in; and in whatever has been added to give them the regular form of a connected story, diligent care has been taken to select such words as might least interrupt the effect of the beautiful English tongue in which he wrote: therefore, words introduced into our language since his time have been as far as possible avoided." 


Q: 
Traduza o parágrafo para o português brasileiro

A: 













































































































































































































































































































In [17]:
#################################################################### RETIRAR REFERÊNCIAS DO TEXTO #################################################################### 

In [2]:
################# Retirar Referências do Texto #################

context = r"""
prompt = '''

Perceba a diferenca entre cada contexto sujo e contexto limpo contido em <<Exemplos>>
Aplique o mesmo raciocinio em <<Resposta>>
Contexto limpo: em <<Resposta>> deve estar limpo de referencias do tipo [algo]

<<Exemplos>>
Contexto sujo: Joao da silva era carpinteiro [1][24]. Estudou na escola santa amelia [17] até os 18 anos de idade.
Contexto limpo: Joao da silva era carpinteiro. Estudou na escola santa amelia até os 18 anos de idade.

Contexto sujo: Ontem foi dia de ramos [32].
Contexto limpo: Ontem foi dia de ramos.
"""

question = """

<<Resposta>>
Contexto sujo:
Ao deixar o país, o imperador D.Pedro I selecionou três pessoas para cuidarem de seu filho e das filhas remanescentes.
A primeira foi José Bonifácio de Andrada, seu amigo e líder influente da independência brasileira, nomeado tutor.[26][27]
A segunda foi Mariana Carlota de Verna Magalhães Coutinho (depois Condessa de Belmonte), que detinha o cargo de aia desde o nascimento de Pedro II.[28]
Quando bebê, D. Pedro II a chamava de "dadama", pois não pronunciava corretamente a palavra "dama".[15]
Considerava-a sua mãe de criação, e continuaria a chamá-la, por afeto, de "dadama" mesmo já adulto.[3][29]
A terceira pessoa escolhida foi Rafael, um veterano negro da Guerra da Cisplatina.[28][30]
Rafael era um empregado do paço em quem D. Pedro I tinha profunda confiança e a quem pediu que olhasse por seu filho — pedido que Rafael levaria a termo pelo resto de sua vida.
José Bonifácio foi destituído de sua posição em dezembro de 1833 e substituído por outro tutor.[31][32][33]
D. Pedro II passava os dias estudando,[34] com apenas duas horas livres para recreação.[35]
Acordava às 06h30 da manhã e começava seus estudos às sete, continuando até as dez da noite, quando ia para cama.[36]
Tomou-se grande cuidado em sua educação para formar valores e personalidade diferente da impulsividade e irresponsabilidade demonstradas por seu pai.[31][37]
Sua paixão pela leitura lhe permitiu assimilar qualquer informação.[38]
D. Pedro II não era um gênio,[39] mas inteligente[40] e com grande capacidade para acumular conhecimento facilmente.[41]
O imperador teve uma infância solitária e infeliz.[8][42]
A perda súbita de seus pais o assombraria por toda a vida;[43] ele teve poucos amigos de sua idade[28][36][44] e o contato com suas irmãs era limitado.[31][34][44]
O ambiente em que foi criado o tornou tímido e carente,[45][46] enxergando nos livros refúgio e fuga do mundo real.

Contexto limpo:
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=1500,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.batch_decode(generation_output, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)


This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.



prompt = '''

Perceba a diferenca entre cada contexto sujo e contexto limpo contido em <<Exemplos>>
Aplique o mesmo raciocinio em <<Resposta>>
Contexto limpo: em <<Resposta>> deve estar limpo de referencias do tipo [algo]

<<Exemplos>>
Contexto sujo: Joao da silva era carpinteiro [1][24]. Estudou na escola santa amelia [17] até os 18 anos de idade.
Contexto limpo: Joao da silva era carpinteiro. Estudou na escola santa amelia até os 18 anos de idade.

Contexto sujo: Ontem foi dia de ramos [32].
Contexto limpo: Ontem foi dia de ramos.


Q: 

<<Resposta>>
Contexto sujo:
Ao deixar o país, o imperador D.Pedro I selecionou três pessoas para cuidarem de seu filho e das filhas remanescentes.
A primeira foi José Bonifácio de Andrada, seu amigo e líder influente da independência brasileira, nomeado tutor.[26][27]
A segunda foi Mariana Carlota de Verna Magalhães Coutinho (depois Condessa de Belmonte), que detinha o cargo de aia desde o nascimento de Pedro II.[28]
Quando bebê, D. Pedro II a chama

In [ ]:
# RESPOSTA COMPLETA

Contexto limpo:

A:

*

*O imperador D. Pedro I selecionou três pessoas para cuidarem de seu filho e das filhas remanescentes.

*A primeira foi José Bonifácio de Andrada, seu amigo e líder influente da independência brasileira, nomeado tutor.

*A segunda foi Mariana Carlota de Verna Magalhães Coutinho (depois Condessa de Belmonte), que detinha o cargo de aia desde o nascimento de Pedro II.

*A terceira pessoa escolhida foi Rafael, um veterano negro da Guerra da Cisplatina.

*Rafael era um empregado do paço em quem D. Pedro I tinha profunda confiança e a quem pediu que olhasse por seu filho — pedido que Rafael levaria a termo pelo resto de sua vida.

*José Bonifácio foi destituído de sua posição em dezembro de 1833 e substituído por outro tutor.

*D. Pedro II passava os dias estudando, com apenas duas horas livres para recreação.

*Tomou-se grande cuidado em sua educação para formar valores e personalidade diferente da impulsividade e irresponsabilidade demonstradas por seu pai.

*Sua paixão pela leitura lhe permitiu assimilar qualquer informação.

*D. Pedro II não era um gênio, mas inteligente e com grande capacidade para acumular conhecimento facilmente.

*O imperador teve uma infância solitária e infeliz.

*A perda súbita de seus pais o assombraria por toda a vida; ele teve poucos amigos de sua idade e o contato com suas irmãs era limitado.

*O ambiente em que foi criado o tornou tímido e carente, enxergando nos livros refúgio e fuga do mundo real.

In [4]:
################# Retirar Referências do Texto #################

context = r"""

Esta é uma tarefa de retirada de referências bibliográficas do texto. Estas referências estão escritas no formato "['número']".

Exemplos:
Contexto: Joao da silva era carpinteiro [1][24]. Estudou na escola santa amelia [17] até os 18 anos de idade.
Resposta: Joao da silva era carpinteiro. Estudou na escola santa amelia até os 18 anos de idade.
Contexto: Ontem foi dia de ramos [32].
Resposta: Ontem foi dia de ramos.
"""



question = """
Contexto: Ao deixar o país, o imperador D.Pedro I selecionou três pessoas para cuidarem de seu filho e das filhas remanescentes. A primeira foi José Bonifácio de Andrada, seu amigo e líder influente da independência brasileira, nomeado tutor.[26][27] A segunda foi Mariana Carlota de Verna Magalhães Coutinho (depois Condessa de Belmonte), que detinha o cargo de aia desde o nascimento de Pedro II.[28] Quando bebê, D. Pedro II a chamava de "dadama", pois não pronunciava corretamente a palavra "dama".[15] Considerava-a sua mãe de criação, e continuaria a chamá-la, por afeto, de "dadama" mesmo já adulto.[3][29] A terceira pessoa escolhida foi Rafael, um veterano negro da Guerra da Cisplatina.[28][30] Rafael era um empregado do paço em quem D. Pedro I tinha profunda confiança e a quem pediu que olhasse por seu filho — pedido que Rafael levaria a termo pelo resto de sua vida.
José Bonifácio foi destituído de sua posição em dezembro de 1833 e substituído por outro tutor.[31][32][33] D. Pedro II passava os dias estudando,[34] com apenas duas horas livres para recreação.[35] Acordava às 06h30 da manhã e começava seus estudos às sete, continuando até as dez da noite, quando ia para cama.[36] Tomou-se grande cuidado em sua educação para formar valores e personalidade diferente da impulsividade e irresponsabilidade demonstradas por seu pai.[31][37] Sua paixão pela leitura lhe permitiu assimilar qualquer informação.[38] D. Pedro II não era um gênio,[39] mas inteligente[40] e com grande capacidade para acumular conhecimento facilmente.[41]
O imperador teve uma infância solitária e infeliz.[8][42] A perda súbita de seus pais o assombraria por toda a vida;[43] ele teve poucos amigos de sua idade[28][36][44] e o contato com suas irmãs era limitado.[31][34][44] O ambiente em que foi criado o tornou tímido e carente,[45][46] enxergando nos livros refúgio e fuga do mundo real.

Retire as referências do texto dado e escreva a Resposta:

"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=1500,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.batch_decode(generation_output, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)




Esta é uma tarefa de retirada de referências bibliográficas do texto. Estas referências estão escritas no formato "['número']".

Exemplos:
Contexto: Joao da silva era carpinteiro [1][24]. Estudou na escola santa amelia [17] até os 18 anos de idade.
Resposta: Joao da silva era carpinteiro. Estudou na escola santa amelia até os 18 anos de idade.
Contexto: Ontem foi dia de ramos [32].
Resposta: Ontem foi dia de ramos.


Q: 
Contexto: Ao deixar o país, o imperador D.Pedro I selecionou três pessoas para cuidarem de seu filho e das filhas remanescentes. A primeira foi José Bonifácio de Andrada, seu amigo e líder influente da independência brasileira, nomeado tutor.[26][27] A segunda foi Mariana Carlota de Verna Magalhães Coutinho (depois Condessa de Belmonte), que detinha o cargo de aia desde o nascimento de Pedro II.[28] Quando bebê, D. Pedro II a chamava de "dadama", pois não pronunciava corretamente a palavra "dama".[15] Considerava-a sua mãe de criação, e continuaria a chamá-la, por af

In [5]:
################# Retirar Referências do Texto #################

context = r"""
Esta é uma tarefa de retirada de referências bibliográficas do texto. Estas referências estão escritas no formato "[número]".

Exemplos:
Contexto: Joao da silva era carpinteiro [1][24]. Estudou na escola santa amelia [17] até os 18 anos de idade.
Resposta: Joao da silva era carpinteiro. Estudou na escola santa amelia até os 18 anos de idade.
Contexto: Ontem foi dia de ramos [32].
Resposta: Ontem foi dia de ramos.
Contexto: Large Language Models (LLMs) have emerged as cutting-edge artificial intelligence systems designed to process and generate text, aiming
to communicate coherently [1]. These developments have brought about a revolutionary transformation [2] by enabling the creation of Large Language Models (LLMs) that can approximate humanlevel performance on certain evaluation benchmarks [3], [4].
Resposta: Large Language Models (LLMs) have emerged as cutting-edge artificial intelligence systems designed to process and generate text, aiming to communicate coherently. These developments have brought about a revolutionary transforbibliográficasmation by enabling the creation of Large Language Models (LLMs) that can approximate humanlevel performance on certain evaluation benchmarks.
"""



question = """
Contexto: Ao deixar o país, o imperador D.Pedro I selecionou três pessoas para cuidarem de seu filho e das filhas remanescentes. A primeira foi José Bonifácio de Andrada, seu amigo e líder influente da independência brasileira, nomeado tutor.[26][27] A segunda foi Mariana Carlota de Verna Magalhães Coutinho (depois Condessa de Belmonte), que detinha o cargo de aia desde o nascimento de Pedro II.[28] Quando bebê, D. Pedro II a chamava de "dadama", pois não pronunciava corretamente a palavra "dama".[15] Considerava-a sua mãe de criação, e continuaria a chamá-la, por afeto, de "dadama" mesmo já adulto.[3][29] A terceira pessoa escolhida foi Rafael, um veterano negro da Guerra da Cisplatina.[28][30] Rafael era um empregado do paço em quem D. Pedro I tinha profunda confiança e a quem pediu que olhasse por seu filho — pedido que Rafael levaria a termo pelo resto de sua vida.
José Bonifácio foi destituído de sua posição em dezembro de 1833 e substituído por outro tutor.[31][32][33] D. Pedro II passava os dias estudando,[34] com apenas duas horas livres para recreação.[35] Acordava às 06h30 da manhã e começava seus estudos às sete, continuando até as dez da noite, quando ia para cama.[36] Tomou-se grande cuidado em sua educação para formar valores e personalidade diferente da impulsividade e irresponsabilidade demonstradas por seu pai.[31][37] Sua paixão pela leitura lhe permitiu assimilar qualquer informação.[38] D. Pedro II não era um gênio,[39] mas inteligente[40] e com grande capacidade para acumular conhecimento facilmente.[41]
O imperador teve uma infância solitária e infeliz.[8][42] A perda súbita dos seus pais o assombraria por toda a vida;[43] ele teve poucos amigos de sua idade[28][36][44] e o contato com suas irmãs era limitado.[31][34][44] O ambiente em que foi criado o tornou tímido e carente,[45][46] enxergando nos livros refúgio e fuga do mundo real.

Retire as referências do texto dado e escreva a Resposta:

"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=1500,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.batch_decode(generation_output, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)


Esta é uma tarefa de retirada de referências bibliográficas do texto. Estas referências estão escritas no formato "[número]".

Exemplos:
Contexto: Joao da silva era carpinteiro [1][24]. Estudou na escola santa amelia [17] até os 18 anos de idade.
Resposta: Joao da silva era carpinteiro. Estudou na escola santa amelia até os 18 anos de idade.
Contexto: Ontem foi dia de ramos [32].
Resposta: Ontem foi dia de ramos.
Contexto: Large Language Models (LLMs) have emerged as cutting-edge artificial intelligence systems designed to process and generate text, aiming
to communicate coherently [1]. These developments have brought about a revolutionary transformation [2] by enabling the creation of Large Language Models (LLMs) that can approximate humanlevel performance on certain evaluation benchmarks [3], [4].
Resposta: Large Language Models (LLMs) have emerged as cutting-edge artificial intelligence systems designed to process and generate text, aiming to communicate coherently. These developme

In [17]:
####################################################################  EXTRAÇÃO DE CONTEÚDO  ####################################################################

In [6]:
################# Reconhecimento e Extração de Entidades #################


context = r"""
O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em várias cores, apresenta um processador mais rápido e maior duração da bateria.
"""

question = """
Analise o parágrafo, identifique Somente as entidades ("entities") dos tipos "pessoa", "lugar" e "organização" mencionadas no texto 
e, em seguida, produza os resultados em formato json.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.batch_decode(generation_output, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)


O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em várias cores, apresenta um processador mais rápido e maior duração da bateria.


Q: 
Analise o parágrafo, identifique Somente as entidades ("entities") dos tipos "pessoa", "lugar" e "organização" mencionadas no texto 
e, em seguida, produza os resultados em formato json.

A: 


























































































































































































































































































































































































































































































































In [7]:
################# Reconhecimento e Extração de Entidades #################


context = r"""
O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em várias cores, apresenta um processador mais rápido e maior duração da bateria.
"""

question = """
Analise o parágrafo, identifique Somente as entities dos tipos "pessoa", "lugar" e "organização" mencionadas no texto e, em seguida, produza os resultados em formato json.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)

# Decodificação e impressão da resposta
answer = tokenizer.batch_decode(generation_output, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)


O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em várias cores, apresenta um processador mais rápido e maior duração da bateria.


Q: 
Analise o parágrafo, identifique Somente as entities dos tipos "pessoa", "lugar" e "organização" mencionadas no texto e, em seguida, produza os resultados em formato json.

A: O parágrafo que você está analisando é o seguinte:


*

*A Apple anunciou o iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana.


O resultado em formato json é:
{
 "entity": "pessoa",
 "type": "organização",
 "value": "Apple"
}


A: O parágrafo que você está analisando é o seguinte:


*

*A Apple anunciou o iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana.


O resultado em formato json é:
{
 "entity": "pessoa",
 "type": "organização",
 "value": "Apple"
}




In [ ]:
# RESPOSTA COMPLETA

A: O parágrafo que você está analisando é o seguinte:

*A Apple anunciou o iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana.


O resultado em formato json é:
{
 "entity": "pessoa",
 "type": "organização",
 "value": "Apple"
}


In [8]:
################# Reconhecimento e Extração de Entidades #################


context = r"""
O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em várias cores, apresenta um processador mais rápido e maior duração da bateria.
"""

question = """
Analise o parágrafo, identifique em formato JSON Somente as entidades ("entities") dos tipos "pessoa", "lugar" e "organização" mencionadas no texto.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)

# Decodificação e impressão da resposta
answer = tokenizer.batch_decode(generation_output, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)


O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em várias cores, apresenta um processador mais rápido e maior duração da bateria.


Q: 
Analise o parágrafo, identifique em formato JSON Somente as entidades ("entities") dos tipos "pessoa", "lugar" e "organização" mencionadas no texto.

A:


{
 "pessoa": [
 {
 "nome": "Tim Cook",
 "sobrenome": "Cook",
 "email": "tim@apple.com",
 "telefone": "+1 408 999 9999",
 "endereço": "1 Infinite Loop, Cupertino, CA 95014",
 "organização": "Apple Inc.",
 "foto": "https://www.apple.com/images/tim-cook.jpg"
 },
 {
 "nome": "Jony Ive",
 "sobrenome": "Ive",
 "email": "jony@apple.com",
 "telefone": "+1 408 999 9999",
 "endereço": "1 Infinite Loop, Cupertino, CA 95014",
 "organização": "Apple Inc.",
 "foto": "https://www.apple.com/images/jony-ive.jpg"
 },
 {
 "nome": "Steve Jobs",
 "sobrenom

In [ ]:
# A resposta completa foi:

A:

{
 "pessoa": [
 {
 "nome": "Tim Cook",
 "sobrenome": "Cook",
 "email": "tim@apple.com",
 "telefone": "+1 408 999 9999",
 "endereço": "1 Infinite Loop, Cupertino, CA 95014",
 "organização": "Apple Inc.",
 "foto": "https://www.apple.com/images/tim-cook.jpg"
 },
 {
 "nome": "Jony Ive",
 "sobrenome": "Ive",
 "email": "jony@apple.com",
 "telefone": "+1 408 999 9999",
 "endereço": "1 Infinite Loop, Cupertino, CA 95014",
 "organização": "Apple Inc.",
 "foto": "https://www.apple.com/images/jony-ive.jpg"
 },
 {
 "nome": "Steve Jobs",
 "sobrenome": "Jobs",
 "email": "steve@apple.com",
 "telefone": "+1 408 999 9999",
 "endereço": "1 Infinite Loop, Cupertino, CA 95014",
 "organização": "Apple Inc.",
 "foto": "https://www.apple.com/images/steve-jobs.jpg"
 },
 {
 "nome": "Jeff Williams",
 "sobrenome": "Williams",
 "email": "jeff@apple.com",
 "telefone": "+1 408 999 9999",
 "endereço": "1 Infinite Loop, Cupertino, CA 95014",
 "organização": "Apple Inc.",
 "foto": "https://www.apple.com/images/jeff-williams.jpg"
 },
 {
 "nome": "Phil Schiller",
 "sobrenome": "Schiller",
 "email": "phil@apple.com",
 "telefone": "+1 408 999 9999",
 "endereço": "1 Infinite Loop, Cupertino, CA 95014",
 "organização": "Apple Inc.",
 "foto": "https://www.apple.com/images/phil-schiller.jpg"
 },
 {
 "nome": "Mike Rockwell",
 "sobrenome": "Rockwell",
 "email": "mike@apple.com",
 "telefone": "+1 408 999 9999",
 "endereço": "1 Infinite Loop, Cupertino, CA 95014",
 "organização": "Apple Inc.",
 "foto": "https://www.apple.com/images/mike-rockwell.jpg"
 },
 {
 "nome": "Kate Adams

In [9]:
################# Extração de Relações #################


context = r"""
Elon Musk fundou a SpaceX em 2002 com o objetivo de reduzir os custos de transporte espacial e possibilitar a colonização de Marte.
"""

question = """
Analise a frase e extraia o relacionamento entre duas entidades e, em seguida, produza os resultados indicando: entities e relatiohships entre estas no formato JSON
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)

# Decodificação e impressão da resposta
answer = tokenizer.batch_decode(generation_output, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)


Elon Musk fundou a SpaceX em 2002 com o objetivo de reduzir os custos de transporte espacial e possibilitar a colonização de Marte.


Q: 
Analise a frase e extraia o relacionamento entre duas entidades e, em seguida, produza os resultados indicando: entities e relatiohships entre estas no formato JSON

A: Você pode usar o seguinte código:
import json

def get_relationships(text):
 entities = []
 relationships = []

 for line in text.splitlines():
 if line.startswith('{'):
 entities.append(line.split(':')[1])
 elif line.startswith('}') or line.startswith('}'):
 relationships.append(line.split(':')[1])

 return entities, relationships

text = '''
{
 "entities": [
 "A",
 "B",
 "C",
 "D",
 "E",
 "F",
 "G",
 "H",
 "I",
 "J",
 "K",
 "L",
 "M",
 "N",
 "O",
 "P",
 "Q",
 "R",
 "S",
 "T",
 "U",
 "V",
 "W",
 "X",
 "Y",
 "Z"
 ],
 "relationships": [
 "A:B",
 "A:C",
 "A:D",
 "A:E",
 "A:F",
 "A:G",
 "A:H",
 "A:I",
 "A:J",
 "A:K",
 "A:L",
 "A:M",
 "A:N",
 "A:O",
 "A:P",
 "A:Q",
 "A:R",
 "A:S",
 "A:T"